In [1]:
import os
import sqlite3

In [2]:
from flask import Flask, request, send_from_directory, \
    render_template, g, redirect, url_for

In [42]:
app = Flask(__name__)
app.config.from_object(__name__)

In [43]:
app.config.update(dict(
        DATABASE=os.path.join(app.root_path, 'flask_test.db')
    ))

In [44]:
def init_db():
    db = sqlite3.connect(app.config['DATABASE'])
    with app.open_resource('schema.sql', mode='r') as f:
        db.cursor().executescript(f.read())
    db.commit()
    db.close()
# init_db()

In [45]:
def connect_db():
    if not hasattr(g, 'db_con'):
        g.db_con = sqlite3.connect(app.config['DATABASE'])
        g.db_con.row_factory = sqlite3.Row
    return g.db_con

In [46]:
@app.teardown_appcontext
def close_db(error):
    if hasattr(g, 'db_con'):
        g.db_con.close()

In [47]:
@app.route('/html/<path:path>')
def send_html(path):
    return send_from_directory('html', path)

In [48]:
@app.route('/save', methods=['POST'])
def save():
    db = connect_db()
    db.execute('insert into entries (username) values (?)', \
               [request.form['username']])
    db.commit()
    return redirect(url_for('list'))

In [49]:
@app.route('/list')
def list():
    db = connect_db()
    cur = db.execute('select username from entries')
    usernames = cur.fetchall()
    return render_template('list.html', users=usernames)

In [50]:
app.run()

teardown
teardown
teardown
teardown
